In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

## [1. Forward Imaging Model](fwdimaging.ipynb)

## 2. The Recovery Process
This process recover the Hi-Res complex object using the Lo-Res image set. Each image of it is generated by the lighting that emerges from a different LED of a grid: the forward imaging process ([fwdimaging.ipynb](fwdimaging.ipynb)).

In [ ]:
from fwdimaging import round_half_up, generate_wave_vectors, generate_input_obj, simulate_lores_img_set
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt

In [ ]:
# Return a list with the indexes of the images for the reconstruction
# sequence or -1 ('int') if the size of the LED grid is not odd in
# which case there is no an unique LED as center.
# The first index is the center. The way is spiral-shaped.
def get_idx_seq_img(grid_len, **kwargs):
    clockwise = kwargs.pop('clockwise', True)
    if grid_len % 2 != 0:
        l_idx_seq_img = list()
        img = int(((grid_len ** 2) - 1) / 2)  #Start in the center.
        l_idx_seq_img.append(img)
        l_addends = [(grid_len, -1), (-grid_len, 1)]
        if not clockwise:
            l_addends = [(grid_len, 1), (-grid_len, -1)]
        # Index to choose between (grid_len, -1) ('tuple') and
        # (-grid_len, 1).
        i = 0
        c = 1
        last_img = (grid_len ** 2) - 1
        while True:
            i %= 2
            v_addend, h_addend = l_addends[i]
            for j in range(c):
                img += v_addend
                if img > last_img:
                    break
                else:
                    l_idx_seq_img.append(img)
            for j in range(c):
                img += h_addend
                if img > last_img:
                    break
                else:
                    l_idx_seq_img.append(img)
            if img >= last_img:
                return l_idx_seq_img
            else:
                i += 1
                c += 1
    else:
        return -1

In [ ]:
def get_photo(**kwargs):
    inpath = kwargs.pop('inpath', '../img-lores/')
    outpath = kwargs.pop('outpath', '../out-img-hires/')
    # Simulate the set of Lo-Res images.
    sim = kwargs.pop('sim', False)
    leds = kwargs.pop('leds', 15)
    loops = kwargs.pop('loops', 5)
    # Show the output Hi-Res image in the notebook.
    show = kwargs.pop('show', True)
    name = kwargs.pop('name', 'out-img-hires')  # Name of the file.
    extension = kwargs.pop('extension', '.tiff')
    show = kwargs.pop('show', '.tiff')
    wvs, _ = generate_wave_vectors()  # wvs by wavevectors.
    if sim:
        in_obj, m, n = generate_input_obj()
        p, q, kx, ky, dkx, dky, cft, img_seq_lores = simulate_lores_img_set(in_obj, m, n, wvs, leds)
    else:  # Import your own Lo-Res image set.
        return -1
    seq = get_idx_seq_img(leds)
    obj_recover = np.ones((m, n), dtype=np.int)
    obj_recover_ft = np.fft.fftshift(np.fft.fft2(obj_recover))
    for i in range(loops):
        for j in range(leds ** 2):
            k = seq[j]
            kxc = round_half_up((n + 1) / 2.0 + kx[k] / dkx)
            kyc = round_half_up((m + 1) / 2.0 + ky[k] / dky)
            kyl = round_half_up(kyc - (p - 1) / 2.0)
            kyh = round_half_up(kyc + (p - 1) / 2.0)
            kxl = round_half_up(kxc - (q - 1) / 2.0)
            kxh = round_half_up(kxc + (q - 1) / 2.0)
            lores_ft = ((p / m) ** 2) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh]
            lores_ft *= cft
            img_lores = np.fft.ifft2(np.fft.ifftshift(lores_ft))
            img_lores = ((m / p) ** 2) * img_seq_lores[k] * np.exp(1j * np.angle(img_lores))
            lores_ft = np.fft.fftshift(np.fft.fft2(img_lores)) * cft
            obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] = (1 - cft) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] + lores_ft
    obj_recover = np.fft.ifft2(np.fft.ifftshift(obj_recover_ft))
    obj_recover = np.absolute(obj_recover)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    scipy.misc.imsave(outpath + name + extension, obj_recover)
    if show:
        plt_img = plt.imshow(obj_recover, cmap='Greys_r')
    return obj_recover

## References
Zheng, G. (2015). *Fourier Ptychographic Imaging: A MATLAB&reg; tutorial*. San Rafael, CA: Morgan &amp; Claypool Publishers.